# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Basic Workflows
Examples copied from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']

Using AI model: openai:gpt-4o


### Workflow: Prompt Chaining

The first and simplest workflow, chaining, we feed the output of one LLM
call to the next one, and complete our task step by step.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> All LLM calls are handled by the `Agent` class. One feature we take advantage of
> here is a consistent interface for different LLM providers. The model name is
> provided to the constructor and from there, the calls are the same regardless of
> which provider and model we are using.

In [5]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        agent = Agent(AI_MODEL, system_prompt=prompt)
        response = await agent.run(f"nInput:\n{result}")
        result = response.output
        show(result, title=f"Step {i}")
    return result

In [6]:
data_processing_steps = [
    """Extract only the numerical values and their associated minion metrics from the text.
    Format each as 'value: minion metric' on a new line.
    Example format:
    92: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Convert all numerical values to percentages where possible for minion analytics.
    If not a percentage or points, convert to decimal (e.g., 92 bananas -> 92%).
    Keep one number per line.
    Example format:
    92%: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Sort all lines in descending order by numerical value for minion performance ranking.
    Keep the format 'value: minion metric' on each line.
    Example:
    92%: banana consumption rate
    87%: yellow outfit compliance""",
    
    """Format the sorted minion data as a markdown table with columns:
    | Minion Metric | Value |
    |:--|--:|
    | Banana Consumption Rate | 92% |"""
]

report = """
Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.
""".strip()

show(report, title="Input text")
formatted_result = await chain(report, data_processing_steps)
show(formatted_result, title="Result")


Input text
----------

Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.


Step 1
------

92: minion happiness score  
45%: "BELLO!" frequency growth  
23%: yellow outfit compliance improvement  
5%: minion distraction incidents  
$43: evil scheme participation cost  
78%: gadget adoption rate  
87: Gru satisfaction  
34%: banana storage efficiency improvement  


Step 2
------

92%: minion happiness score  
45%: "BELLO!" frequency growth  
23%: yellow outfit compli

### Workflow: Routing

In the routing workflow, we rely on an LLM call to decide which
of several options to take. An AI agent is now handling part of
the logic of the app.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> One of the most powerful features of Pydantic AI is ... Pydantic!
> Returing results using structured outputs and validating them against
> the schema specified is built-in. We can specify a `result_type`
> for every agent and receive the parsed and validated Pydantic model
> instance from the LLM call. This is easily one of the best ways of
> interfacing between code and LLM calls, and we'll be using it for
> most workflows.

In [12]:
class RouteSelection(BaseModel):
    reasoning: str = Field(..., description=(
        'Brief explanation of why this ticket should be routed '
        'to a specific team. Consider key terms, user intent, '
        'and urgency level.'
    ))
    selection: str = Field(..., description='The chosen team name')


async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    # First, determine appropriate route using LLM with chain-of-thought
    show(f"{list(routes.keys())}", title="Available Routes")
    
    routing_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team '
            f'from these options: {list(routes.keys())}'
        ),
        output_type=RouteSelection,
    )
    route_response = await routing_agent.run(input)
    reasoning = route_response.output.reasoning
    route_key = route_response.output.selection.strip().lower()
    
    show(reasoning, title="Routing Analysis")
    show(f"{route_key}", title="Selected Route")
    
    # Process input with selected specialized prompt
    worker_agent = Agent(AI_MODEL, system_prompt=routes[route_key])
    return (await worker_agent.run(input)).output

In [13]:
support_routes = {
    "banana": """You are a banana supply specialist at Gru's lab. Follow these guidelines:
    1. Always start with "Banana Supply Response:"
    2. First acknowledge the specific banana-related issue
    3. Explain banana inventory, quality, or delivery problems clearly
    4. List concrete next steps with timeline for banana resolution
    5. End with alternative banana options if relevant
    
    Keep responses enthusiastic but professional about banana matters.""",
    
    "gadget": """You are a minion gadget technical support engineer. Follow these guidelines:
    1. Always start with "Gadget Support Response:"
    2. List exact steps to resolve the gadget malfunction
    3. Include safety requirements for minion operation
    4. Provide workarounds for common gadget problems
    5. End with escalation path to Dr. Nefario if needed
    
    Use clear, numbered steps and mention "BELLO!" for encouragement.""",
    
    "security": """You are a minion security specialist for Gru's operations. Follow these guidelines:
    1. Always start with "Security Support Response:"
    2. Prioritize lab security and minion identification verification
    3. Provide clear steps for access recovery/changes to restricted areas
    4. Include security tips and warnings about minion imposters
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone while being minion-friendly.""",
    
    "training": """You are a minion training specialist. Follow these guidelines:
    1. Always start with "Training Support Response:"
    2. Focus on minion skill development and best practices
    3. Include specific examples of proper minion behavior
    4. Link to relevant training manual sections
    5. Suggest related skills that might help with evil schemes
    
    Be educational and encouraging, use minion language occasionally."""
}

# Test with different minion support tickets
tickets = [
    """Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin""",
    
    """Subject: Banana shortage in cafeteria
    Message: Hello, I just noticed we're completely out of bananas in the minion cafeteria, but I 
    thought we had a fresh shipment scheduled for today. The other minions are getting restless 
    without their banana breaks. Can you explain this shortage and fix it?
    Thanks,
    Minion Stuart""",
    
    """Subject: How to operate the new shrink ray?
    Message: I need to learn how to operate the new shrink ray gadget for tomorrow's mission. I've 
    looked through the manual but can't figure out the safety protocols. Is there a training 
    session available? Could you walk me through the steps?
    Best regards,
    Minion Bob"""
]

print("Processing minion support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    show(ticket, title=f"Ticket {i}")
    response = await route(ticket, support_routes)
    show(response, title=f"Response {i}")

Processing minion support tickets...


Ticket 1
--------

Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin


Available Routes
----------------

['banana', 'gadget', 'security', 'training']


Routing Analysis
----------------

Kevin is experiencing an access issue with a scanner that likely concerns security clearance or recognition within a secured area (the secret lab). The usage of the term "unrecognized minion error" suggests that this could be related to access privileges or identity recognition, which falls under the purview of the security team. The urgency is further heightened by the need to prepare the freeze ray by the end of the day, indicating a time-sensitive security issue.


Selected Route
-

### Workflow: Parallelization

LLM calls are typically long-running, I/O-bound operations. They are also stateless,
so in cases where one call doesn't depend on the other, it makes a lot of sense
to parallelize the calls and continue processing once we receive the responses
from all of them.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> `Agent.run()` is an asynchronous call, and in the code here is asnchronous by default.
> Pydantic AI also provides an `Agent.run_sync()` method for cases where you're writing
> synchronous code.


In [17]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(AI_MODEL, system_prompt=prompt)
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.output for result in results]

In [18]:
stakeholders = [
    """Minions:
    - Banana supply dependent
    - Want latest gadgets
    - Safety concerns with explosions""",
    
    """Lab Scientists (Dr. Nefario):
    - Equipment reliability worries
    - Need advanced technology
    - Want clear evil scheme direction""",
    
    """Gru (Boss):
    - Expects successful missions
    - Want cost-effective operations
    - Reputation risks with failed schemes""",
    
    """Gadget Suppliers:
    - Production capacity constraints
    - Price pressures from competition
    - Technology upgrade transitions"""
]

impact_results = await parallel(
    """Analyze how changes in the evil villain industry will impact this stakeholder group.
    Provide specific impacts and recommended actions for Gru's operations.
    Format with clear sections and priorities.""",
    stakeholders
)

for stakeholder, result in zip(stakeholders, impact_results):
    show(result, stakeholder.split(':')[0])


Minions
-------

**Impact Analysis on Minions**

1. **Banana Supply Dependency**

   *Impact:*
   - As the evil villain industry modernizes and potentially moves towards synthetic and sustainable resources, there may be shifts in how staples such as bananas are sourced.
   - Supply chain disruptions due to increased regulatory oversight or environmental considerations could lead to shortages.

   *Recommended Actions:*
   - Prioritize establishing secure, sustainable banana supply lines, including the possibility of investing in banana plantations or setting up partnerships with global suppliers.
   - Develop contingency plans, such as exploring alternative food sources that meet the caloric and nutritional needs of Minions, to mitigate dependency.

   *Priority: High*  
   Securing a stable food supply is essential for maintaining Minion productivity and morale.

2. **Desire for Latest Gadgets**

   *Impact:*
   - As technological advancements accelerate, Minions may demand access to